<h2><p style=' color:DeepSkyBlue;' >PDB PREPARATION FOR ALPHA GALACTOSE A PROTEIN</h2>

ENV:htmd  

### Sections
[Common part](#common)  
[APO only build](#apo)  
[DGJ only build](#holo-(DGJ))  

In [ ]:
from htmd.ui import *
import pandas as pd 
import numpy as np
import os 
from htmd.builder import charmm
from moleculekit.tools.graphalignment import maximalSubstructureAlignment
from acemd.protocols import setup_equilibration
from acemd.protocols import setup_production
import re 



#DATA THAT CAN BE CHANGED
folder = f'3GXT_reglyco' #prova


molecule = 'glycosylation/reglyco/3gxt_reglyco.pdb' #from RCSB.org or local pdb (in this case, glycoshape.org)
mutations = [('resid 215', 'SER')] # #resid to mutate, if present, wt is not produced #,('resid 301', 'GLN')

resnames_to_remove = ['SO4', 'HOH', 'NOJ'] #if nothing to remove ignore
#glycan patches
patch = ['patch NGLB P0:139 P2:2', 'patch NGLB P0:192 P3:2', 'patch NGLB P0:215 P4:2',
         'patch NGLB P1:139 P5:2', 'patch NGLB P1:192 P6:2', 'patch NGLB P1:215 P7:2',
         'patch 14BB P2:2 P2:3', 'patch 14BB P3:2 P3:3', 'patch 14BB P4:2 P4:3', 'patch 14BB P5:2 P5:3', 'patch 14BB P6:2 P6:3', 'patch 14BB P7:2 P7:3',
         'patch 14BB P2:3 P2:4', 'patch 14BB P3:3 P3:4', 'patch 14BB P4:3 P4:4', 'patch 14BB P5:3 P5:4', 'patch 14BB P6:3 P6:4', 'patch 14BB P7:3 P7:4',
         'patch 13AB P2:4 P2:6', 'patch 13AB P3:4 P3:6', 'patch 13AB P4:4 P4:6', 'patch 13AB P5:4 P5:6', 'patch 13AB P6:4 P6:6', 'patch 13AB P7:4 P7:6',
         'patch 16AT P2:4 P2:5', 'patch 16AT P3:4 P3:5', 'patch 16AT P4:4 P4:5', 'patch 16AT P5:4 P5:5', 'patch 16AT P6:4 P6:5', 'patch 16AT P7:4 P7:5',
         ] #top/top_all36_carb.rtf

gly_resid = [139, 192, 215] #glycosilation sites

#SIMULATION DATA
eq_run = '50 ns' #also in us, ns, ps and fs
eq_temp = 300
minimize = 1000
prod_run = '1 us' #also in us, ns, ps and fs
prod_temp = 300

#correct ligand for Holo str
dgj_a = 'DGJ/DGJ_A.cgenff.mol2' #chain A 
dgj_b = 'DGJ/DGJ_B.cgenff.mol2' #chain B

#MAKE FOLDER FOR {MOLECULE}
os.makedirs(folder, exist_ok=True) #check and make

#AMINO ACID CODES FOR CONVERSION (do not change)
aa_3to1 = {'ALA': 'A', 'CYS': 'C', 'ASP': 'D', 'GLU': 'E', 'PHE': 'F', 'GLY': 'G', 
 'HIS': 'H', 'ILE': 'I', 'LYS': 'K', 'LEU': 'L', 'MET': 'M', 'ASN': 'N', 
 'PRO': 'P', 'GLN': 'Q', 'ARG': 'R', 'SER': 'S', 'THR': 'T', 'VAL': 'V', 
 'TRP': 'W', 'TYR': 'Y'} #convertion from 3 letter code to 1 letter code

2025-06-24 14:41:10,187 - numexpr.utils - INFO - Note: detected 128 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
2025-06-24 14:41:10,188 - numexpr.utils - INFO - Note: NumExpr detected 128 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
2025-06-24 14:41:10,188 - numexpr.utils - INFO - NumExpr defaulting to 16 threads.



Please cite HTMD: Doerr et al.(2016)JCTC,12,1845. https://dx.doi.org/10.1021/acs.jctc.6b00049
HTMD Documentation at: https://software.acellera.com/htmd/

New HTMD version (2.5.6) is available. You are currently on (2.5.5).
We recommend you create a new conda environment with the latest HTMD version.
Run: `conda create -n htmd2.5.6 htmd=2.5.6 -c acellera -c conda-forge`




2025-06-24 14:41:12,995 - acemd - WARNING - New ACEMD version (4.0.16) is available. You are currently on (4.0.13). We recommend you create a new conda environment with the latest ACEMD version https://software.acellera.com/acemd/install.html 



Acellera Software Not-for-Profit License Agreement v1.2

The software ("Software") has been developed by the contributing
researchers from Acellera and made available through Acellera for your
internal, non-profit research use.

Acellera allows researchers at your institution to run, display, copy and
modify Software on the following conditions:

1. The Software remains at your institution and is not published, distributed,
   or otherwise transferred or made available to other than institution employees
   and students involved in research under your supervision.

2. You agree to make results generated using Software available to other
   academic researchers for non-profit research purposes. If you wish to obtain
   Software for any commercial purposes, including fee-based service projects, you
   will need to execute a separate licensing agreement with Acellera and pay a
   fee. In that case please contact: info@acellera.com.

3. You retain in Software and any modifications to Soft

## Common 
[Back to main](#sections) 

In [ ]:
mol = Molecule(molecule)
#REMOVE RESNAMES
print(f'\nRemoving residues: {resnames_to_remove}') #check
original_count = mol.numAtoms
selection = ' or '.join([f'resname {res}' for res in resnames_to_remove]) #build correct sele
mol.filter(f'not ({selection})') #remove
removed_count = original_count - mol.numAtoms #check
print(f'Removed {removed_count} atoms')
print(f'Remaining atoms: {mol.numAtoms}')

#CREATE DUPLICATE FOR MOL_APO 
mol_apo = mol.copy()
mol_DGJ = mol.copy()


Removing residues: ['SO4', 'HOH', 'NOJ']
Removed 0 atoms
Remaining atoms: 6961


## APO
[Back to main](#sections) 

In [ ]:
#HANDLE MUTATIONS
proteins_apo=[]

if mutations: #form is <resid  resid> and <mut resname>
    for mutation in mutations:
        resid_sel, new_res = mutation 
        wt = np.unique(mol_apo.get('resname', resid_sel))
        wt = aa_3to1[wt[0]] 
        num = resid_sel.split(' ')[1]
        mut = aa_3to1[new_res]

        folder_apo = f'{folder}/apo_{wt}{num}{mut}'
        os.makedirs(folder_apo, exist_ok=True)
        
        #MAKE A COPY OF THE APO TO BE MUTATED + STORE DATA
        mol_apo_mut = mol_apo.copy()
        mol_apo_mut.mutateResidue(resid_sel, new_res)

        #CHECK IF MUTATION IN GLYCOSILATION SITE
        segid_to_remove = []

        if int(num) in gly_resid:
            print(f'MUTATION IN GLYCOSILATION SITE:{wt}{num}{mut}')
            new_patch = []
            ##segid_to_remove=[] #as for the atoms but iterative

            #remove patches and glycans where mutation occours
            for p in patch:
                if re.search(rf'\b{num}\b', p): #match resid
                    print(f'REMOVING PROTEIN PATCH {p}')
                    match = re.search(rf'{num}\s+(\w+):', p)
                    if match:
                        segid = match.group(1) #get resid to remove
                        segid_to_remove.append(segid)
                        print(f'SEGID TO BE REMOVED: {segid}') 
                    continue 
                if any(segid in p for segid in segid_to_remove): #remove any other patches related to segid
                    print(f'REMOVING GLYCAN PATCHES {p}')
                    continue 
                else: #if glycan not attached to mutation
                    new_patch.append(p)
            patch = new_patch #original name, works with wild type too       
            
            #save segid to remove
            sel = ' or '.join([f'segid {sgd}' for sgd in segid_to_remove]) #sgd=segid
            mol_apo_mut.filter(f'not ({sel})')
            print(segid_to_remove)
        proteins_apo.append((f'apo_{wt}{num}{mut}', mol_apo_mut, folder_apo, segid_to_remove))


#FOR WILD TYPE
else: #no mutations
    folder_apo = f'{folder}/apo'
    os.makedirs(folder_apo, exist_ok=True) 
    proteins_apo.append(('apo', mol_apo, folder_apo, []))

#COMMON
for label, mol, folder_apo, segid_to_remove in proteins_apo:
    print(f"\nProcessing system: {label}")

    #SYSTEM PREPARATION AND SEGMENTATION.
    #return *_prep.csv (residue protonation info), *_prep.pdb, *_pka.png
    #additional parameters: titration, hydrophobic_thickness, force_protonation, no_opt, no_prot, no_titr, hold_nonpeptidic_bonds, ignore_ns_errors, residue_smiles
    system_apo, data = systemPrepare(mol, pH=7.0, return_details=True, plot_pka=f'{folder_apo}/{folder}_{label}_pka')
    #Detects resid gaps in a selection and assigns incrementing segid to each fragment (there is also autoSegment2)
    system_apo = autoSegment(system_apo) #segment system
    
    #remove segid of glycans if in mutation site
    if segid_to_remove:
        sel = ' or '.join([f'segid {sgd}' for sgd in segid_to_remove])
        system_apo.filter(f'not ({sel})')
        print(f"REMOVED GLYCANS AT: {sel}")

    #intermediate saving
    data.to_csv(f'{folder_apo}/{folder}_{label}_prep.csv') #save
    system_apo.write(f'{folder_apo}/{folder}_{label}_prep.pdb') #save

    segments_apo = np.unique(system_apo.segid) #check
    print(f'Segments: {segments_apo}')

    #SYSTEM SOLVATION
    system_solv_apo = solvate(system_apo, negx = 20  , negy = 20, negz = 20, posx = 20, posy = 20, posz = 20)
    system_solv_apo.write(f'{folder_apo}/{folder}_{label}_solv.pdb')
    #system_solv.write(f'{folder_apo}/{molecule}_solv.psf') #non giusto

    #SYSTEM BUILDING WITH CHARMM36 AND PATCHES
    #other parameters available
    system_charmm_apo = charmm.build(system_solv_apo,  saltconc = 0.15, saltanion = 'CL', saltcation = 'K',
                                topo= ['top/top_all36_prot.rtf', 'top/top_all36_carb.rtf', 'top/top_water_ions.rtf', 'top/top_all36_cgenff.rtf', 'DGJ/top_DGJ.rtf'],    
                                param=['par/par_all36m_prot.prm','par/par_all36_carb.prm','par/par_water_ions.prm', 'par/par_all36_cgenff.prm', 'DGJ/par_DGJ.prm'],
                                stream=['str/carb/toppar_all36_carb_glycopeptide.str'],
                                patches = patch,  
                                outdir = f'{folder_apo}/build') #patches = patch,

#apo ma equilibration non produce xsc per boxsize (giustamente)
#MINIMIZATION AND EQUILIBRATION PREPARATION
setup_equilibration(builddir=f'{folder_apo}/build', 
                        outdir=f'{folder_apo}/equilibration',
                        run = eq_run, #also in us, ns, ps and fs
                        temperature = eq_temp,
                        coordinates = f'{folder_apo}/build/structure.pdb',
                        structure = f'{folder_apo}/build/structure.psf',
                        parameters = f'{folder_apo}/build/parameters.prm',
                        minimize = minimize)


#to remember:
# "NA","MG","ZN","K","CS","CA","CL"  
#, 'noj/noj_g.rtf'
#charmm.listFiles()  #check for files  
# 


#### The **production** folder can be generated only **after the production is compleded**.

In particular:

1. check production ended with *check_end.py* 
2. run *production_prep.py*

## HOLO (DGJ)
[Back to main](#sections) 

In [3]:
#HANDLE MUTATIONS
proteins_DGJ=[]

if mutations:
    for mutation in mutations: #form is <resid  resid> and <mut resname>
        resid_sel, new_res = mutation
        wt = np.unique(mol_DGJ.get('resname', resid_sel))
        wt = aa_3to1[wt[0]]
        num = resid_sel.split(' ')[1]
        mut = aa_3to1[new_res]
        
        folder_DGJ = f'{folder}/DGJ_{wt}{num}{mut}'
        os.makedirs(folder_DGJ, exist_ok=True)
        
        #MAKE A COPY OF THE DGJ TO BE MUTATED + STORE DATA
        mol_DGJ_mut = mol_DGJ.copy()
        mol_DGJ_mut.mutateResidue(resid_sel, new_res)       

        #CHECK IF MUTATION IN GLYCOSILATION SITE
        segid_to_remove = []

        if int(num) in gly_resid:
            print(f'MUTATION IN GLYCOSILATION SITE:{wt}{num}{mut}')
            new_patch = []
            
            #remove patches and glycans where mutation occours
            for p in patch:
                if re.search(rf'\b{num}\b', p): #match resid
                    print(f'REMOVING PROTEIN PATCH {p}')
                    match = re.search(rf'{num}\s+(\w+):', p)
                    if match:
                        segid = match.group(1) #get resid to remove
                        segid_to_remove.append(segid)
                        print(f'SEGID TO BE REMOVED: {segid}') 
                    continue 
                if any(segid in p for segid in segid_to_remove): #remove any other patches related to segid
                    print(f'REMOVING GLYCAN PATCHES {p}')
                    continue 
                else: #if glycan not attached to mutation
                    new_patch.append(p)
            patch = new_patch #original name, works with wild type too  

            #save segid to remove
            sel = ' or '.join([f'segid {sgd}' for sgd in segid_to_remove]) #sgd=segid
            mol_DGJ_mut.filter(f'not ({sel})')
            print(segid_to_remove)
        proteins_DGJ.append((f'DGJ_{wt}{num}{mut}', mol_DGJ_mut, folder_DGJ, segid_to_remove))    

#FOR WILD TYPE
else: #no mutations
    folder_DGJ = f'{folder}/DGJ'
    os.makedirs(folder_DGJ, exist_ok=True)
    proteins_DGJ.append(('DGJ', mol_DGJ, folder_DGJ, []))

#COMMON
for label, mol, folder_DGJ, segid_to_remove in proteins_DGJ: 
    print(f"\nProcessing system: {label}")

    #APPEND CORRECT DGJ IN CHAIN A AND B
    #chain A
    DGJ_A = Molecule(dgj_a)
    DGJ_A.set('resid', '1', 'resname DGJ')
    DGJ_A.set('chain', 'L', 'resname DGJ')
    #chain B
    DGJ_B = Molecule(dgj_b)
    DGJ_B.set('resid', '2', 'resname DGJ')
    DGJ_B.set('chain', 'L', 'resname DGJ')
    #append
    mol_DGJ.append(DGJ_A)
    mol_DGJ.append(DGJ_B)

    #SYSTEM PREPARATION AND SEGMENTATION.
    #return *_prep.csv (residue protonation info), *_prep.pdb, *_pka.png
    #additional parameters: titration, hydrophobic_thickness, force_protonation, no_opt, no_prot, no_titr, hold_nonpeptidic_bonds, ignore_ns_errors, residue_smiles
    system_DGJ, data = systemPrepare(mol_DGJ, pH=7.0, return_details=True, plot_pka=f'{folder_DGJ}/_{folder}_{label}_pka')
    #Detects resid gaps in a selection and assigns incrementing segid to each fragment (there is also autoSegment2)
    system_DGJ = autoSegment(system_DGJ) #segment system

    #remove segid of glycans in mutation site
    if segid_to_remove:
        sel = ' or '.join([f'segid {sgd}' for sgd in segid_to_remove])
        system_DGJ.filter(f'not ({sel})')
        print(f"REMOVED GLYCANS AT: {sel}")

    #intermediate saving
    data.to_csv(f'{folder_DGJ}/{folder}_{label}_prep.csv') #save
    system_DGJ.write(f'{folder_DGJ}/{folder}_{label}_prep.pdb') #save

    segments_DGJ = np.unique(system_DGJ.segid) #check
    print(f'Segments: {segments_DGJ}')

    #SYSTEM SOLVATION
    system_solv_DGJ = solvate(system_DGJ, negx = 20  , negy = 20, negz = 20, posx = 20, posy = 20, posz = 20)
    system_solv_DGJ.write(f'{folder_DGJ}/{folder}_{label}_solv.pdb')
    #system_solv.write(f'{folder}/_solv.psf') #non giusto

    #SYSTEM BUILDING WITH CHARMM36 AND PATCHES
    #other parameters available
    system_charmm_DGJ = charmm.build(system_solv_DGJ,  saltconc = 0.15, saltanion = 'CL', saltcation = 'K',
                                topo= ['top/top_all36_prot.rtf', 'top/top_all36_carb.rtf', 'top/top_water_ions.rtf', 'top/top_all36_cgenff.rtf', 'DGJ/top_DGJ.rtf'],    
                                param=['par/par_all36m_prot.prm','par/par_all36_carb.prm','par/par_water_ions.prm', 'par/par_all36_cgenff.prm', 'DGJ/par_DGJ.prm'],
                                stream=['str/carb/toppar_all36_carb_glycopeptide.str'],
                                patches = patch,
                                outdir = f'{folder_DGJ}/build') 

    #to remember:
    # "NA","MG","ZN","K","CS","CA","CL'  
    #, 'noj/noj_g.rtf'
    #charmm.listFiles()  #check for files                          

    #MINIMIZATION AND EQUILIBRATION PREPARATION
    setup_equilibration(builddir=f'{folder_DGJ}/build', 
                        outdir=f'{folder_DGJ}/equilibration',
                        run = eq_run, 
                        temperature = eq_temp,
                        coordinates = f'{folder_DGJ}/build/structure.pdb',
                        structure = f'{folder_DGJ}/build/structure.psf',
                        parameters = f'{folder_DGJ}/build/parameters.prm',
                        minimize = minimize)


MUTATION IN GLYCOSILATION SITE:N215S
REMOVING PROTEIN PATCH patch NGLB P0:215 P4:2
SEGID TO BE REMOVED: P4
REMOVING PROTEIN PATCH patch NGLB P1:215 P7:2
SEGID TO BE REMOVED: P7
REMOVING GLYCAN PATCHES patch 14BB P4:2 P4:3
REMOVING GLYCAN PATCHES patch 14BB P7:2 P7:3
REMOVING GLYCAN PATCHES patch 14BB P4:3 P4:4
REMOVING GLYCAN PATCHES patch 14BB P7:3 P7:4
REMOVING GLYCAN PATCHES patch 13AB P4:4 P4:6
REMOVING GLYCAN PATCHES patch 13AB P7:4 P7:6
REMOVING GLYCAN PATCHES patch 16AT P4:4 P4:5
REMOVING GLYCAN PATCHES patch 16AT P7:4 P7:5
['P4', 'P7']

Processing system: DGJ_N215S


2025-06-24 14:41:21,392 - moleculekit.tools.preparation - WARNING - Both chains and segments are defined in Molecule.chain / Molecule.segid, however they are inconsistent. Protein preparation will use the chain information.



---- Molecule chain report ----
Chain A:
    First residue: LEU    32  
    Final residue: MET   421  
Chain B:
    First residue: LEU    32  
    Final residue: GLN   422  
Chain C:
    First residue: NAG     2  
    Final residue: MAN     6  
Chain D:
    First residue: NAG     2  
    Final residue: MAN     6  
Chain E:
    First residue: NAG     2  
    Final residue: MAN     6  
Chain F:
    First residue: NAG     2  
    Final residue: MAN     6  
Chain G:
    First residue: NAG     2  
    Final residue: MAN     6  
Chain H:
    First residue: NAG     2  
    Final residue: MAN     6  
Chain L:
    First residue: DGJ     1  
    Final residue: DGJ     2  
---- End of chain report ----



2025-06-24 14:41:22,104 - moleculekit.tools.preparation - INFO - Found 6 covalent bonds from protein to non-protein molecules.
2025-06-24 14:41:22,105 - moleculekit.tools.preparation - INFO - Freezing protein residue ASN:A:139 bonded to non-protein molecule NAG:C:2
2025-06-24 14:41:22,105 - moleculekit.tools.preparation - INFO - Freezing protein residue ASN:A:192 bonded to non-protein molecule NAG:D:2
2025-06-24 14:41:22,105 - moleculekit.tools.preparation - INFO - Freezing protein residue ASN:A:215 bonded to non-protein molecule NAG:E:2
2025-06-24 14:41:22,106 - moleculekit.tools.preparation - INFO - Freezing protein residue ASN:B:139 bonded to non-protein molecule NAG:F:2
2025-06-24 14:41:22,106 - moleculekit.tools.preparation - INFO - Freezing protein residue ASN:B:192 bonded to non-protein molecule NAG:G:2
2025-06-24 14:41:22,106 - moleculekit.tools.preparation - INFO - Freezing protein residue ASN:B:215 bonded to non-protein molecule NAG:H:2
2025-06-24 14:41:29,438 - moleculekit.t

REMOVED GLYCANS AT: segid P4 or segid P7
Segments: ['P0' 'P1' 'P2' 'P3' 'P5' 'P6' 'P8']


2025-06-24 14:41:37,746 - htmd.builder.solvate - INFO - Using water pdb file at: /leonardo/home/userexternal/icazzani/miniconda3/envs/htmd/lib/python3.10/site-packages/htmd/share/solvate/wat.pdb
2025-06-24 14:41:38,194 - htmd.builder.solvate - INFO - Replicating 18 water segments, 3 by 2 by 3
Solvating: 100%|██████████| 18/18 [00:07<00:00,  2.40it/s]
2025-06-24 14:41:51,393 - htmd.builder.solvate - INFO - 61835 water molecules were added to the system.
2025-06-24 14:41:51,550 - moleculekit.writers - WARNING - Field "serial" of PDB overflows. Your data will be truncated to 5 characters.
2025-06-24 14:41:58,158 - htmd.builder.charmm - INFO - Writing out segments.
2025-06-24 14:42:06,563 - moleculekit.writers - WARNING - Field "serial" of PDB overflows. Your data will be truncated to 5 characters.
2025-06-24 14:42:07,313 - moleculekit.writers - WARNING - Field "serial" of PDB overflows. Your data will be truncated to 5 characters.
2025-06-24 14:42:07,853 - moleculekit.writers - WARNING - 

Disulfide Bond between: UniqueResidueID<resname: 'CYX', chain: 'B', resid: 56, insertion: '', segid: 'P1'>
                   and: UniqueResidueID<resname: 'CYX', chain: 'B', resid: 63, insertion: '', segid: 'P1'>

Disulfide Bond between: UniqueResidueID<resname: 'CYX', chain: 'A', resid: 56, insertion: '', segid: 'P0'>
                   and: UniqueResidueID<resname: 'CYX', chain: 'A', resid: 63, insertion: '', segid: 'P0'>

Disulfide Bond between: UniqueResidueID<resname: 'CYX', chain: 'A', resid: 52, insertion: '', segid: 'P0'>
                   and: UniqueResidueID<resname: 'CYX', chain: 'A', resid: 94, insertion: '', segid: 'P0'>

Disulfide Bond between: UniqueResidueID<resname: 'CYX', chain: 'A', resid: 378, insertion: '', segid: 'P0'>
                   and: UniqueResidueID<resname: 'CYX', chain: 'A', resid: 382, insertion: '', segid: 'P0'>

Disulfide Bond between: UniqueResidueID<resname: 'CYX', chain: 'B', resid: 52, insertion: '', segid: 'P1'>
                   and: UniqueR

2025-06-24 14:42:14,922 - htmd.builder.charmm - INFO - Starting the build.
2025-06-24 14:42:15,998 - htmd.builder.charmm - WARNING - Failed to set coordinates for 150 atoms.
2025-06-24 14:42:15,998 - htmd.builder.charmm - WARNING - Failed to guess coordinates for 6 atoms due to bad angles.
2025-06-24 14:42:15,999 - htmd.builder.charmm - WARNING - Poorly guessed coordinates for 38 atoms.
2025-06-24 14:42:15,999 - htmd.builder.charmm - WARNING - Please check /leonardo/home/userexternal/icazzani/aGAL/3GXT_reglyco/DGJ_N215S/build/log.txt for further information.
2025-06-24 14:42:15,999 - htmd.builder.charmm - INFO - Finished building.
2025-06-24 14:42:18,153 - moleculekit.readers - WARNING - Non-integer values were read from the PDB "serial" field. Dropping PDB values and assigning new ones.
2025-06-24 14:42:18,236 - moleculekit.readers - WARNING - Reading PDB file with more than 99999 atoms. Bond information can be wrong.
2025-06-24 14:42:23,960 - htmd.builder.ionize - INFO - Adding 0 ani

#### The **production** folder can be generated only **after the production is compleded**.

In particular:

1. check production ended with *check_end.py* 
2. run *production_prep.py*

IGNORARE